# Metadata Application Profile Assignment

In [30]:
import json
import requests

In [31]:
def get_credential(json_file, key, sub_key):
    try:
        with open(json_file) as f:
            data = json.load(f)
            return data[key][sub_key]
    except Exception as e:
        print("Error: ", e)

In [32]:
base_url = 'http://emmaheck.projectst.si.umich.edu/omekas/omeka-s/api'
key_identity = get_credential("secrets.json", "omeka", "key_identity")
key_credential = get_credential("secrets.json", "omeka", "key_credential")
print(key_identity)
print(key_credential)

9KiOiA9e800PCOgf8ypxQALZ5PwqtuaK
hGALPZ7kQqs12K4q6Jymw8UNsgLeNgiH


In [33]:
omekas_credentials = {
    'key_identity': key_identity,
    'key_credential': key_credential
}

In [34]:
import csv
import json
import requests

# for later, when working with local files
import glob
import os
from os.path import join

In [56]:
endpoint = 'https://www.loc.gov/free-to-use'
parameters = {
    'fo' : 'json'
}

In [57]:
collection = "cats"

In [58]:
collection_list_response = requests.get(endpoint + '/' + collection, params=parameters)

In [59]:
collection_list_response.url

'https://www.loc.gov/free-to-use/cats?fo=json'

In [60]:
collection_json = collection_list_response.json()

In [61]:
collection_json.keys()

dict_keys(['breadcrumbs', 'content', 'content_is_post', 'description', 'expert_resources', 'next', 'next_sibling', 'options', 'pages', 'portal', 'previous', 'previous_sibling', 'site_type', 'timestamp', 'title', 'type'])

In [62]:
for k in collection_json['content']['set']['items']:
    print(k)

{'image': '/static/portals/free-to-use/public-domain/cats/16.jpg', 'link': '/resource/ppmsca.15528/', 'title': 'Edward Penfield, his book. Bookplate, ca. 1900-1925'}
{'image': '/static/portals/free-to-use/public-domain/cats/1.jpg', 'link': '/resource/pga.12042/', 'title': 'The cat - Felis domesticus. Lithograph, published by L. Prang, 1872.'}
{'image': '/static/portals/free-to-use/public-domain/cats/2.jpg', 'link': '/resource/cph.3c21044/', 'title': 'Animal locomotion, Twenty-four consecutive images of a cat running. Photos by Eadweard Muybridge, 1887'}
{'image': '/static/portals/free-to-use/public-domain/cats/3.jpg', 'link': '/resource/jpd.02425/', 'title': 'Domestic cat nursing kittens. Ukiyo-e woodcut by Tachibana Morikuni, 1720.'}
{'image': '/static/portals/free-to-use/public-domain/cats/4.jpg', 'link': '/resource/jpd.02798/', 'title': 'A cat cleaning its claws. Drawing by Katsushika Hokusai, between 1830 and 1850'}
{'image': '/static/portals/free-to-use/public-domain/cats/5.jpg', 

In [63]:
len(collection_json['content']['set']['items'])

40

In [64]:
collection_json['content']['set']['items'][0].keys()

dict_keys(['image', 'link', 'title'])

In [65]:
collection_set_list = os.path.join('data','collection_set_list.csv')
headers = ['image','link','title']

with open(collection_set_list, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for item in collection_json['content']['set']['items']:
        # clean up errant spaces in the title fields
        item['title'] = item['title'].rstrip()
        writer.writerow(item)
    print('wrote',collection_set_list)

wrote data/collection_set_list.csv


In [66]:
# update endpoint info
endpoint = 'https://www.loc.gov'
parameters = {
    'fo' : 'json'
}

In [67]:
# run this cell to confirm that you have a location for the JSON files
item_metadata_directory = os.path.join('data','ftu_libs_metadata')

if os.path.isdir(item_metadata_directory):
    print(item_metadata_directory,'exists')
else:
    os.mkdir(item_metadata_directory)
    print('created',item_metadata_directory)

created data/ftu_libs_metadata


In [69]:
item_count = 0
error_count = 0
file_count = 0

data_directory = 'data'
item_metadata_directory = 'ftu_libs_metadata'
item_metadata_file_start = 'item_metadata'
json_suffix = '.json'

collection_set_list = os.path.join('data','collection_set_list.csv')

with open(collection_set_list, 'r', encoding='utf-8', newline='') as f:
    reader = csv.DictReader(f, fieldnames=headers)
    for item in reader:
        if item['link'] == 'link':
            continue
        # these resource links could redirect to item pages, but currently don't work
        if '?' in item['link']:
            resource_ID = item['link']
            short_ID = item['link'].split('/')[2]
            item_metadata = requests.get(endpoint + resource_ID + '&fo=json')
            print('requested',item_metadata.url,item_metadata.status_code)
            if item_metadata.status_code != 200:
                print('requested',item_metadata.url,item_metadata.status_code)
                error_count += 1
                continue
            try:
                item_metadata.json()
            except: #basically this catches all of the highsmith photos with hhh in the ID
                error_count += 1
                print('no json found')
                continue
            fout = os.path.join(data_directory, item_metadata_directory, str(item_metadata_file_start + '-' + short_ID + json_suffix))
            with open(fout, 'w', encoding='utf-8') as json_file:
                json_file.write(json.dumps(item_metadata.json()['item']))
                file_count += 1
                print('wrote', fout)
            item_count += 1
        else:
            resource_ID = item['link']
            short_ID = item['link'].split('/')[2]
            item_metadata = requests.get(endpoint + resource_ID, params=parameters)
            print('requested',item_metadata.url,item_metadata.status_code)
            if item_metadata.status_code != 200:
                print('requested',item_metadata.url,item_metadata.status_code)
                error_count += 1
                continue
            try:
                item_metadata.json()
            except:
                error_count += 1
                print('no json found')
                continue
            fout = os.path.join(data_directory, item_metadata_directory, str(item_metadata_file_start + '-' + short_ID + json_suffix))
            with open(fout, 'w', encoding='utf-8') as json_file:
                json_file.write(json.dumps(item_metadata.json()['item']))
                file_count += 1
                print('wrote', fout)
            item_count += 1

print('--- mini LOG ---')
print('items requested:',item_count)
print('errors:',error_count)
print('files written:',file_count)

requested https://www.loc.gov/resource/ppmsca.15528/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-ppmsca.15528.json
requested https://www.loc.gov/resource/pga.12042/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-pga.12042.json
requested https://www.loc.gov/resource/cph.3c21044/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-cph.3c21044.json
requested https://www.loc.gov/resource/jpd.02425/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-jpd.02425.json
requested https://www.loc.gov/resource/jpd.02798/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-jpd.02798.json
requested https://www.loc.gov/resource/cph.3b06249/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-cph.3b06249.json
requested https://www.loc.gov/resource/cph.3g08753/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-cph.3g08753.json
requested https://www.loc.gov/resource/pga.05046/?fo=json 200
wrote data/ftu_libs_metadata/item_metadata-pga.05046.json
requested https://www.

In [70]:
current_loc = os.getcwd()

print(current_loc)

/Users/emmaheck/Documents/umich/courses/SI676/networked-services-labs-2023-main


In [71]:
metadata_file_path = os.path.join('data','ftu_libs_metadata')

print(metadata_file_path)


data/ftu_libs_metadata


In [72]:
file_count = 0

for file in glob.glob('data/ftu_libs_metadata/item_metadata-*.json'):
    file_count += 1
    print(file)
    
print('found',file_count)

data/ftu_libs_metadata/item_metadata-agc.7a17761.json
data/ftu_libs_metadata/item_metadata-cph.3c06978.json
data/ftu_libs_metadata/item_metadata-agc.7a16795.json
data/ftu_libs_metadata/item_metadata-ppmsca.43507.json
data/ftu_libs_metadata/item_metadata-ds.07446.json
data/ftu_libs_metadata/item_metadata-cph.3g08217.json
data/ftu_libs_metadata/item_metadata-jpd.02425.json
data/ftu_libs_metadata/item_metadata-jpd.02798.json
data/ftu_libs_metadata/item_metadata-npcc.10856.json
data/ftu_libs_metadata/item_metadata-ds.04037.json
data/ftu_libs_metadata/item_metadata-cph.3c00170.json
data/ftu_libs_metadata/item_metadata-highsm.47959.json
data/ftu_libs_metadata/item_metadata-rbc0001.2003juv0001.json
data/ftu_libs_metadata/item_metadata-cph.3g08753.json
data/ftu_libs_metadata/item_metadata-ppmsca.18772.json
data/ftu_libs_metadata/item_metadata-pga.07149.json
data/ftu_libs_metadata/item_metadata-pga.12042.json
data/ftu_libs_metadata/item_metadata-pga.05046.json
data/ftu_libs_metadata/item_metada

In [73]:
list_of_item_metadata_files = list() 
for file in glob.glob('data/ftu_libs_metadata/item_metadata-*.json'):
    list_of_item_metadata_files.append(file)

In [74]:
len(list_of_item_metadata_files)

39

In [75]:
list_of_item_metadata_files.sort()

for file in list_of_item_metadata_files:
    print(file)

data/ftu_libs_metadata/item_metadata-agc.7a16795.json
data/ftu_libs_metadata/item_metadata-agc.7a17761.json
data/ftu_libs_metadata/item_metadata-cai.2a15065.json
data/ftu_libs_metadata/item_metadata-cph.3b06249.json
data/ftu_libs_metadata/item_metadata-cph.3b16490.json
data/ftu_libs_metadata/item_metadata-cph.3b39357.json
data/ftu_libs_metadata/item_metadata-cph.3c00170.json
data/ftu_libs_metadata/item_metadata-cph.3c06978.json
data/ftu_libs_metadata/item_metadata-cph.3c20459.json
data/ftu_libs_metadata/item_metadata-cph.3c21044.json
data/ftu_libs_metadata/item_metadata-cph.3g03063.json
data/ftu_libs_metadata/item_metadata-cph.3g03065.json
data/ftu_libs_metadata/item_metadata-cph.3g08217.json
data/ftu_libs_metadata/item_metadata-cph.3g08753.json
data/ftu_libs_metadata/item_metadata-ds.04034.json
data/ftu_libs_metadata/item_metadata-ds.04037.json
data/ftu_libs_metadata/item_metadata-ds.07446.json
data/ftu_libs_metadata/item_metadata-fsa.8c01765.json
data/ftu_libs_metadata/item_metadata-

In [76]:
# try first with one file, can you open the json, can you see what elements are in the json?
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as item:
    # what are we looking at?
    print('file:',list_of_item_metadata_files[0],'\n')
    
    # load the item data
    item_data = json.load(item)
    
    for element in item_data.keys():
        print(element,':',item_data[element])

file: data/ftu_libs_metadata/item_metadata-agc.7a16795.json 

_version_ : 1731246297655017472
access_restricted : False
aka : ['https://www.loc.gov/pictures/item/2018706861/', 'http://www.loc.gov/item/2018706861/', 'http://www.loc.gov/pictures/item/2018706861/', 'https://www.loc.gov/pictures/collection/agc/item/2018706861/', 'http://www.loc.gov/pictures/collection/agc/item/2018706861/', 'https://hdl.loc.gov/loc.pnp/agc.7a16795', 'https://hdl.loc.gov/loc.pnp/agc.7a16795', 'http://www.loc.gov/resource/agc.7a16795/', 'http://lccn.loc.gov/2018706861']
call_number : LC-G432- 0187 [P&P]
campaigns : []
contributor_names : ['Genthe, Arnold, 1869-1942, photographer']
contributors : [{'genthe, arnold': 'https://www.loc.gov/search/?fa=contributor:genthe,+arnold&fo=json'}]
control_number : 
created : 2019-10-22T06:20:22Z
created_published : ['ca. 1912.']
created_published_date : ca. 1912.
date : 1912-01-01
dates : [{'1912': 'https://www.loc.gov/search/?dates=1912/1912&fo=json'}]
description : ['1 

In [77]:
item_data.keys()

dict_keys(['_version_', 'access_restricted', 'aka', 'call_number', 'campaigns', 'contributor_names', 'contributors', 'control_number', 'created', 'created_published', 'created_published_date', 'date', 'dates', 'description', 'digital_id', 'digitized', 'display_offsite', 'extract_timestamp', 'extract_urls', 'format', 'format_headings', 'genre', 'group', 'hassegments', 'id', 'image_url', 'index', 'item', 'language', 'languages', 'library_of_congress_control_number', 'link', 'marc', 'medium', 'medium_brief', 'mime_type', 'modified', 'notes', 'number', 'number_former_id', 'number_lccn', 'number_source_modified', 'online_format', 'original_format', 'other_control_numbers', 'other_formats', 'other_title', 'partof', 'related', 'repository', 'reproduction_number', 'reproductions', 'resource_links', 'resources', 'rights', 'rights_advisory', 'rights_information', 'score', 'shard', 'shelf_id', 'site', 'sort_date', 'source_collection', 'source_created', 'source_modified', 'subject', 'subject_headi

In [93]:
print('\ncontributors:', item_data['contributor_names'])

IndexError: list index out of range

In [79]:
# can you get the date?
print('\ndate:',item_data['date'], type(item_data['date']))
    # can you get the format?
print('\nformat:',item_data['format'][0], type(item_data['format']))


date: 1912-01-01 <class 'str'>

format: {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'} <class 'list'>


In [80]:
# set up the containers to create the csv of all the item fields
# file for csv to read out
collection_info_csv = 'collection_items_data.csv'

# set up a list for the columns in your csv; 
# your goal should be to automate this, but . . . 
# it works for demonstration as you set up the crosswalk
headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'dig_format', 'rights']

# try first with one file
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    # load the item data
    item_data = json.load(data)
    
    # extract the data you want
    # for checking purposes, add in the source of the info
    source_file = str(file)
    # make sure there's some unique and stable identifier
    try:
        item_id = item_data['library_of_congress_control_number']
    except:
        item_id = item_data['url'].split('/')[-2]
    title = item_data['title']
    date = item_data['date']
    source_url = item_data['url']
    try:
        phys_format = item_data['format'][0]
    except:
        phys_format = 'Not found'
    try:
        dig_format = item_data['online_format'][0]
    except:
        dig_format = 'Not found'
    mime_type = item_data['mime_type']
    try:
        rights = item_data['rights_information']
    except:
        rights = 'Undetermined'


    # dictionary for the rows
    row_dict = dict()
    
    # look for the item metadata, assign it to the dictionary; 
    # start with some basic elements likely (already enumerated in the headers list) :
    # source file
    row_dict['source_file'] = source_file
    # identifier
    row_dict['item_id'] = item_id
    # title
    row_dict['title'] = title
    # date
    row_dict['date'] = date
    # link
    row_dict['source_url'] = source_url
    # format
    row_dict['phys_format'] = phys_format
    # digital format
    row_dict['dig_format'] = dig_format
    #rights
    row_dict['rights'] = rights 
    print('created row dictionary:',row_dict)

    # write to the csv
    with open(collection_info_csv, 'w', encoding='utf-8') as fout:
        writer = csv.DictWriter(fout, fieldnames=headers)
        writer.writeheader()
        writer.writerow(row_dict)
        print('wrote',collection_info_csv)

created row dictionary: {'source_file': 'data/ftu_libs_metadata/item_metadata-thc.5a36851.json', 'item_id': '2018706861', 'title': 'Buzzer the cat, portrait photograph', 'date': '1912-01-01', 'source_url': 'https://www.loc.gov/item/2018706861/', 'phys_format': {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'}, 'dig_format': 'image', 'rights': 'No known restrictions on publication. For information, see: "Genthe photograph collection" https://hdl.loc.gov/loc.pnp/res.073.agc'}
wrote collection_items_data.csv


In [83]:
collection_info_csv = 'collection_items_data.csv'

# set up a list for the columns in your csv; in future, this should be more automated but this works for now as you set up the crosswalk
headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'dig_format', 'rights']

# try first with one file
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    # load the item data
    item_data = json.load(data)
    
    print(item_data['image_url'][2])

https://tile.loc.gov/storage-services/service/pnp/agc/7a16000/7a16700/7a16795r.jpg#h=392&w=560


In [85]:
from datetime import date

date_string_for_today = date.today().strftime('%Y-%m-%d') # see https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior

print(date_string_for_today)


2023-11-20


In [99]:
# set up the containers to create the csv & counters 
# file for csv to read out
collection_info_csv = os.path.join('data','collection_items_data.csv')
file_count = 0
items_written = 0
error_count = 0

# add in a couple of extras for Omeka, including item type and date uploaded

# set up a list for the columns in your csv; in future, this should be more automated but this works for now as you set up the crosswalk
headers = ['title', 'date', 'description', 'call_number', 'contributor_names', 'format', 'language', 'mime_type', 'subject', 'partof']

# now, adapt the previous loop to open each file:
for file in list_of_item_metadata_files:
    file_count += 1
    print('opening',file)
    with open(file, 'r', encoding='utf-8') as item:
        # load the item data
        try:
            item_data = json.load(item)
        except:
            print('error loading',file)
            error_count += 1
            continue

        # extract/name the data you want
        # item type
        item_type = 'Item'
        # date uplaoded
        date_uploaded = date_string_for_today
        # for checking purposes, add in the source of the info
        source_file = str(file)
        # make sure there's some unique and stable identifier
        try:
            title = item_data['title']
        except:
            title = 'not found'
        try:
            date = item_data['date']
        except:
            date = 'Not found'
        try:
            description = item_data['description']
        except:
            description = 'Not found'
        try:
            call_number = item_data['call_number']
        except:
            call_number = 'not found'
        try:
            contributor_names = item_data['contributor_names']
        except:
            contributor_names = 'not found'
        try:
            format = item_data['format']
        except:
            format = 'not found'
        try:
            language = item_data['language']
        except:
            language = 'not found'
        try:
            mime_type = item_data['mime_type']
        except:
            mime_type = 'not found'
        try:
            subject = item_data['subject']
        except:
            subject = 'not found'
        try:
            partof = item_data['partof'][0]
        except:
            partof = 'not found'
        # dictionary for the rows
        row_dict = dict()

        # look for the item metadata, assign it to the dictionary; 
        # start with some basic elements likely (already enumerated in the headers list) :
        # title
        row_dict['title'] = title
        # date
        row_dict['date'] = date
        # description
        row_dict['description'] = description
        # call num
        row_dict['call_number'] = call_number
        # creator
        row_dict['contributor_names'] = contributor_names
        # original format
        row_dict['format'] = format
        # language
        row_dict['language'] = language
        # mime type
        row_dict['mime_type'] = mime_type
        # subject
        row_dict['subject'] = subject
        # part of
        row_dict['partof'] = partof


        # write to the csv
        with open(collection_info_csv, 'a', encoding='utf-8') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers)
            if items_written == 0:
                writer.writeheader()
            writer.writerow(row_dict)
            items_written += 1
            print('adding',item_id)

print('\n\n--- LOG ---')
print('wrote',collection_info_csv)
print('with',items_written,'items')
print(error_count,'errors (info not written)')


opening data/ftu_libs_metadata/item_metadata-agc.7a16795.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-agc.7a17761.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cai.2a15065.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3b06249.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3b16490.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3b39357.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3c00170.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3c06978.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3c20459.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3c21044.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3g03063.json
adding 2018706861
opening data/ftu_libs_metadata/item_metadata-cph.3g03065.json
adding 2018706861
opening data/ftu_libs_metadata/item_meta